In [1]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [2]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [3]:
display(df_train.head())
display(df_test.head())

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍


,index,class,title
0,1800,體育,如果騎士火箭進入總決賽，誰的勝算大？
1,1801,體育,從個人競技狀態來看，三個階段的詹姆斯，哪個最強？
2,1802,體育,騎士總冠軍！地球人誰能阻擋詹姆斯？史上最佳就是他！打服所有人
3,1803,體育,詹姆斯絕殺，騎士3比0，猛龍懷疑人生
4,1804,體育,騎士和步行者戰成搶七險勝，而猛龍即將被橫掃，步行者跟猛龍的區別在哪裡？


In [4]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [5]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [6]:
# 忽略警告
import warnings
warnings.simplefilter("ignore")
ws, pos = WS('./data/'), POS('./data/')

In [7]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [8]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [9]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# 排除較無意義的詞性

In [10]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:50])
    print('=======================')

Nb : ['切沃', '恆生', '魯能', '高曉松', '霍金斯', '張大仙', '艾科麥佛鯊', '騰訊王', '舒斯特爾', '德比', '大神', '單薇恩', '富力', '伯克希爾', '戈麥斯', '泰倫盧', '朱嘯虎', '布茲德里克', '建成', '里坦克', '阿里巴巴', '章澤天', '電一', '庫里', '金英權', '小白', '伏爾加格勒', '馬龍', '小明', '昌圖', '勒布朗', '東皇', '米切爾西蒙斯', '盼保', 'P60八核讓', '拉卡拉', '刷安徒恩', '阿迪薩亞', '惠若琪', '紫鑫', '王楚欽', '圍甲', '哪吒', '萊昂納德', '梁建章', '旭旭', '昂科威', '安琪拉', '喬丹', 'U23']
VC : ['玩不動', '破除', '拍拍', '報', '獨造', '衝施', '丟棄', '秀', '比較', '實現', '拿', '購置', '留下', '舉辦', '學會', '操作', '問倒', '打不過', '找回', '過上', '追', '帶動', '審查', '監控', '毀滅', '止損', '帶火', '降', '撿到', '裝', '實施', '照看', '拋棄', '挑選出', '玩', '開出', '實拍', '嫁', '翻', '包車', '覽', '套現', '趕', '接', '拉黑', '制裁', '下調', '考', '遇到', '送走']
Na : ['葷素', '貨源', '有錢人', '參數', '閃電狼', '聖水', '騙子', '商人', '國家級', '莊稼漢', '星星', '花式', '頭腦', '水質', '原油', '老爸', '幣值', '夢奇', '紅心', '搏擊', '季後賽', '獎勵', '日', '腦', '電量', '命運', '戒指', '銀', '水怪', '人物', '背', '職業', '藥劑', '野螳螂', '答案', '媒體', '可口可樂', '雞蛋', '主網', '大型', '年費', '萬達學', '葬禮', '愛', '樣子', '遺憾', '段位', '女', '風潮', '目標']
COLONCATEGORY : ['：', ':']
Nc : ['韓國', '自

|         Type        |     Description    |         Type        |     Description    |         Type        |     Description    |         Type        |     Description    |         Type        |     Description    |
|:-------------------:|:------------------:|:-------------------:|:------------------:|:-------------------:|:------------------:|:-------------------:|:------------------:|:-------------------:|:------------------:|
| A                   | 非謂形容詞         | Na                  | 普通名詞           | Nv                  | 名物化動詞         | VHC                 | 狀態使動動詞       | COLONCATEGORY       | 冒號               |
| Caa                 | 對等連接詞         | Nb                  | 專有名詞           | P                   | 介詞               | VI                  | 狀態類及物動詞     | COMMACATEGORY       | 逗號               |
| Cab                 | 連接詞，如：等等   | Nc                  | 地方詞             | T                   | 語助詞             | VJ                  | 狀態及物動詞       | DASHCATEGORY        | 破折號             |
| Cba                 | 連接詞，如：的話   | Ncd                 | 位置詞             | VA                  | 動作不及物動詞     | VK                  | 狀態句賓動詞       | DOTCATEGORY         | 點號               |
| Cbb                 | 關聯連接詞         | Nd                  | 時間詞             | VAC                 | 動作使動動詞       | VL                  | 狀態謂賓動詞       | ETCCATEGORY         | 刪節號             |
| D                   | 副詞               | Nep                 | 指代定詞           | VB                  | 動作類及物動詞     | V_2                 | 有                 | EXCLAMATIONCATEGORY | 驚嘆號             |
| Da                  | 數量副詞           | Neqa                | 數量定詞           | VC                  | 動作及物動詞       |                     |                    | PARENTHESISCATEGORY | 括號               |
| Dfa                 | 動詞前程度副詞     | Neqb                | 後置數量定詞       | VCL                 | 動作接地方賓語動詞 |                     |                    | PAUSECATEGORY       | 頓號               |
| Dfb                 | 動詞後程度副詞     | Nes                 | 特指定詞           | VD                  | 雙賓動詞           | DE                  | 的之得地           | PERIODCATEGORY      | 句號               |
| Di                  | 時態標記           | Neu                 | 數詞定詞           | VF                  | 動作謂賓動詞       | SHI                 | 是                 | QUESTIONCATEGORY    | 問號               |
| Dk                  | 句副詞             | Nf                  | 量詞               | VE                  | 動作句賓動詞       | FW                  | 外文               | SEMICOLONCATEGORY   | 分號               |
| DM                  | 定量式             | Ng                  | 後置詞             | VG                  | 分類動詞           |                     |                    | SPCHANGECATEGORY    | 雙直線             |
| I                   | 感嘆詞             | Nh                  | 代名詞             | VH                  | 狀態不及物動詞     |                     |                    | WHITESPACE          | 空白               |

In [11]:
# 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
def get_excluded_flags(train_title_cuts, apper_times=20):
    pos_analysis = {}
    flags=[]
    
    for _, pairs in train_title_cuts.items():
        for word, flag in pairs:
            if flag not in pos_analysis:
                pos_analysis[flag] = set()
            pos_analysis[flag].add(word)

    for flag, words in pos_analysis.items():
        if len(words) < apper_times:
            flags.append(flag)
            
    return flags

In [12]:
def get_selection_train_title_cuts(train_title_cuts):
    excluded_flags = get_excluded_flags(train_title_cuts)
    selection_train_title_cuts = {}
    
    for idx, pairs in train_title_cuts.items():
        _list = []
        for pair in pairs:
            if pair[1] not in excluded_flags:
                _list.append(pair)
        selection_train_title_cuts[idx] =_list
        
    return selection_train_title_cuts

# Bag of Words (BOW)

In [13]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
unique_words = list(set(word for pairs in train_title_cuts.values() for word, _ in pairs))

for index, word in enumerate(unique_words):
    word2index[word] = index
    index2word[index] = word

In [14]:
print(word2index['溫暖'])
print(index2word[word2index['溫暖']])

1873
溫暖


In [15]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(len(word2index))
    
    for word, _ in pairs:
        if word in word2index :
            vector[word2index[word]] += 1
            
    return vector

In [16]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = get_excluded_flags(train_title_cuts)
    vector = np.zeros(len(word2index))
    
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
            
    return vector

# Cosine Similarity

In [17]:
def cosine_similarity(bow1, bow2):
    uni_dist = lambda x: x/(np.sqrt(np.sum(x**2)))
    return np.dot(uni_dist(bow1),uni_dist(bow2))

In [18]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [19]:
display(train_title_cuts[100])
display(train_title_cuts[130])

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [20]:
group_vectors = {news_class: [] for news_class in all_news_class}
group_mean_vector = {}

for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
    
display(group_mean_vector)

{'體育': array([0.        , 0.        , 0.00333333, ..., 0.00333333, 0.        ,
        0.        ]),
 '財經': array([0.  , 0.  , 0.  , ..., 0.  , 0.02, 0.  ]),
 '科技': array([0.        , 0.        , 0.        , ..., 0.        , 0.00666667,
        0.        ]),
 '旅遊': array([0.        , 0.        , 0.        , ..., 0.        , 0.01333333,
        0.        ]),
 '農業': array([0.00333333, 0.        , 0.        , ..., 0.        , 0.01333333,
        0.00333333]),
 '遊戲': array([0.        , 0.00333333, 0.        , ..., 0.        , 0.00333333,
        0.        ])}

# Group mean vector: 測試

In [21]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [22]:
from collections import Counter

accuracy=[]

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    prediciton=round(counter[group]/sum(counter.values())*100,2)
    accuracy.append(prediciton)
    print(f'{group} : {str(counter):70} \taccuracy : {prediciton}%')

print(f'\nAverage accuracy : {round(np.mean(accuracy),2)}%')

體育 : Counter({'體育': 71, '遊戲': 9, '財經': 6, '旅遊': 4, '農業': 3, '科技': 1})       	accuracy : 75.53%
財經 : Counter({'財經': 67, '科技': 18, '農業': 9, '旅遊': 6, '遊戲': 6, '體育': 3})      	accuracy : 61.47%
科技 : Counter({'科技': 64, '財經': 13, '體育': 8, '農業': 7, '遊戲': 3, '旅遊': 2})      	accuracy : 65.98%
旅遊 : Counter({'旅遊': 71, '農業': 11, '財經': 6, '遊戲': 3, '科技': 2, '體育': 1})      	accuracy : 75.53%
農業 : Counter({'農業': 65, '旅遊': 7, '科技': 5, '體育': 4, '財經': 3, '遊戲': 1})       	accuracy : 76.47%
遊戲 : Counter({'遊戲': 78, '體育': 12, '科技': 9, '旅遊': 8, '財經': 5, '農業': 4})      	accuracy : 67.24%

Average accuracy : 70.37%
